In [1]:
import importlib
import gym_model
importlib.reload(gym_model)
from visualisation import JupyterVizGym, GymDrawer
from optimization import LayoutTemplate

In [4]:
template = LayoutTemplate.circular(height=20, width=6)
import random

model_params = {
    "interarrival_time": {
        "type": "SliderInt",
        "value": 5,
        "label": "time between arrivals:",
        "min": 1,
        "max": 50,
        "step": 1,
    },
    "machine_density": {
        "type": "SliderFloat",
        "value": 0.25,
        "label": "Density of gym equipment:",
        "min": 0.05,
        "max": 1,
        "step": 0.05,
    },
    # "layout": template.instantiate([random.choice(list(gym_model.Equipment)) for _ in range(len(template))]),
    # "spawn_location": template.entrance,
}

page = JupyterVizGym(
    gym_model.Gym,
    model_params,
    measures=["Searching"],
    name="Test gym",
    agent_portrayal=lambda agent: agent.portrayal,
    # space_drawer="default",
    space_drawer=GymDrawer,
)

page

Cannot show ipywidgets in text